<a href="https://colab.research.google.com/github/hamzamusht550/TODOList/blob/main/Perfect_FFmpeg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

#@title Install Dependencies

from IPython.display import clear_output
from google.colab import drive
from google.colab import auth

!apt-get install -y mediainfo ffmpeg mkvtoolnix aria2
!pip install patool pymediainfo ffmpeg-python pyrogram tgcrypto nest_asyncio

drive.mount('/content/drive')
auth.authenticate_user()
clear_output()

In [ ]:

#@title Get Mediainfo MKV

MKV_INFO = "/content/The.Roshans.S01.720p.NF.WEB-DL.AAC5.1.AV1-PrimeFix/E2 (1).mkv" #@param {type: "string"}

from IPython.display import clear_output
clear_output()

# !ffmpeg -i {MKV_INFO}
# !ffprobe -i "{MKV_INFO}" -show_streams
!mediainfo "{MKV_INFO}" && mediainfo "{MKV_INFO}" > "{MKV_INFO}".txt
!du -h "{MKV_INFO}"
!du -BM "{MKV_INFO}"

In [ ]:


#@title FFmpeg

FILES = "/content/The.Roshans.S01.720p.NF.WEB-DL.AAC5.1.AV1-PrimeFix/E2.mkv"  # @param {type: "string"}
FFMPEG_OPTIONS = "-map 0:v -map 0:a:m:language:hin -map 0:s? -c copy"  #@param {type: "string"}
Delete_Extracted_Data = False  #@param {type: "boolean"}
#@markdown ***Start 00:00:00 / End 00:00:10***
trim_start = ""  #@param {type: "string"}
trim_end = ""  #@param {type: "string"}
JUST_DOWNLOAD= False  #@param {type: "boolean"}
JUST_EXTRACT= False  #@param {type: "boolean"}

from IPython.display import clear_output
import subprocess
import os
import re
import requests
import patoolib
from google.colab import drive
from zipfile import ZipFile
from pymediainfo import MediaInfo
import mimetypes
# Mount Google Drive
drive.mount('/content/drive')
clear_output()


def strToList(listStr, separator=","):
    listStr = ' '.join(listStr.split()) # Remove extra spaces from String
    myList = [_.strip() for _ in listStr.split(separator)]
    myList = [_ for _ in myList if _ and _.strip()] # Remove empty items from list
    showAllURLs = "\n".join(f"{i+1}. {ele}" for i, ele in enumerate(myList))
    print(showAllURLs)
    return myList


def printObj(obj):
    objToStr = str(vars(obj))
    print(objToStr)


def getSize(input_file):
    !du -h "{input_file}"
    !du -BM "{input_file}"


def copy(source, destination, isDeleteSource=0):
    if isDeleteSource == 0:
        !rsync -ah --progress "{source}" "{destination}"
    if isDeleteSource == 1:
        !rsync -ah --progress --remove-source-files "{source}" "{destination}"
        print(f"Source deleted: {source}")
    print(f"Transfer Successfull:\nSource => {source}\nDestination => {destination}")


def deleteFile(input_file):
    !rm "{input_file}"
    print(f"File Deleted: {input_file}")


def deleteFolder(input_folder):
    !rm -rf "{input_folder}"
    print(f"Folder Deleted: {input_folder}")


def printMediaInfo(input_file):
    """Display media information."""
    !mediainfo "{input_file}"


def extract_file_info(file_path):
    # parent_dir_path = os.path.dirname(file_path)
    # file_name = os.path.basename(file_path)
    parent_dir_path, file_name = os.path.split(file_path)
    _name, _ext = os.path.splitext(file_name)
    return parent_dir_path,  file_name, _name, _ext[1:]

def add_suffix_to_file(file_path, suffix="_Suffix"):
    parent_dir_path, file_name = os.path.split(file_path)
    name, ext = os.path.splitext(file_name)
    new_file_name = f"{name}{suffix}{ext}"
    new_file_path = os.path.join(parent_dir_path, new_file_name)
    return new_file_path

def filter_existing_filepath(base_path):
    directory, filename = os.path.split(base_path)
    name, ext = os.path.splitext(filename)

    counter = 1
    new_path = base_path
    while os.path.exists(new_path):
        new_name = f"{name} ({counter}){ext}" if ext else f"{name} ({counter})"
        new_path = os.path.join(directory, new_name)
        counter += 1

    return new_path


def getFilesList(folder_path, relative=0):
    all_files = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            full_path = os.path.join(root, file)
            if relative == 1:
                full_path = os.path.relpath(full_path, folder_path)
            all_files.append(full_path)

    all_files.sort()
    return all_files


def runCommand(cmd, showLog=0):
    if showLog == 1:
        # Print logs in real-time, if 2nd param is 1
        print(f"Running command: {cmd}")
        process = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
        for line in iter(process.stdout.readline, b''):
            print(line.decode("utf-8"), end="")

        process.stdout.close()
        process.wait()
    else:
        process = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    return process.returncode


def getResponseFileName(url):
    # Define the maximum filename length (commonly 255 characters)
    MAX_FILENAME_LENGTH = 255

    # Get the response from the URL
    response = requests.get(url, stream=True)

    # Extract the filename from the Content-Disposition header
    filename = response.headers.get('content-disposition', '')
    if 'filename=' in filename:
        filename = filename.split("filename=")[-1].strip('"')
    else:
        return ""

    # Define a regex pattern to match unsuitable characters for filenames
    invalid_chars = r'[<>:"/\\|?*\']'

    # Replace unsuitable characters with a hyphen
    valid_filename = re.sub(invalid_chars, '-', filename)

    # Truncate the filename if it exceeds the maximum length
    if len(valid_filename) > MAX_FILENAME_LENGTH:
        # Preserve the file extension if present
        if '.' in valid_filename:
            name, extension = valid_filename.rsplit('.', 1)
            truncated_name = name[:MAX_FILENAME_LENGTH - len(extension) - 1]  # Reserve space for the dot
            valid_filename = f"{truncated_name}.{extension}"
        else:
            valid_filename = valid_filename[:MAX_FILENAME_LENGTH]

    return valid_filename


def Aria2Downloader(url, output_file=''):
    if not url.startswith("http"):
        print(f"Invalid URL: {url}")
    _responseFileName = getResponseFileName(url)
    if _responseFileName:
        if not output_file:
            output_file = _responseFileName
        if os.path.exists(output_file):
            print(f"Download File Already Exists: {output_file}")
            return output_file
        else:
            """Download a file using Aria2."""
            print(f"Downloading file: {url}")
            command = f"aria2c -x 16 -s 16 -o \"{output_file}\" {url}"
            status_code = runCommand(command, 1)
            if status_code == 0:
                print(f"\nDownload Successfull: {output_file}\n")
                return output_file
            else:
                print(f"Download Error Status code({status_code}): {url}")
    else:
        print(f"Download Error Response FileName: {url}")
    return url


def useFFmpeg(input_file, output_file='', log=0):
    if not output_file:
        output_file = input_file
    output_file = filter_existing_filepath(output_file)
    global FFMPEG_OPTIONS
    global trim_start
    global trim_end
    trim = ""
    if trim_start:
        trim = f"-ss {trim_start} -to {trim_end}"
    if not FFMPEG_OPTIONS:
        FFMPEG_OPTIONS = "-map 0:v -map 0:a:m:language:hin -map 0:s? -c copy"
    command = f"ffmpeg -i \"{input_file}\" {trim} {FFMPEG_OPTIONS} -y \"{output_file}\""
    runCommand(command, log)
    print("Original Size:")
    getSize(input_file)
    print("NEW Size:")
    getSize(output_file)
    return output_file

def useFFmpegFolder(input_folder, log=0):
    ffmpeg_folder = input_folder + "_ffmpeg"
    if os.path.exists(ffmpeg_folder):
        deleteFolder(ffmpeg_folder)
    os.makedirs(ffmpeg_folder, exist_ok=True)

    for i, file in enumerate(getFilesList(input_folder)):
        if file.endswith(('.mkv', '.mp4', '.avi')):
            output_file = os.path.join(ffmpeg_folder, os.path.split(file)[1])
            print(f"{i+1}: {file}")
            useFFmpeg(file, output_file, log)

    print(f"\nFFmpeg Folder Operation Success: {ffmpeg_folder}\n")
    return ffmpeg_folder


def extractArchive(Archive_File, extracted_folder=''):
    if not extracted_folder:
        extracted_folder = os.path.splitext(Archive_File)[0]
    if os.path.exists(extracted_folder):
        deleteFolder(extracted_folder)
    os.makedirs(extracted_folder, exist_ok=True)
    print(f"\nExtracting archive to: {extracted_folder}")
    patoolib.extract_archive(Archive_File, outdir=extracted_folder)
    print(f"Extraction complete: {extracted_folder}\n")
    return extracted_folder

def createArchive(_folder, Archive_filepath):
    if os.path.exists(Archive_filepath):
        Archive_filepath = filter_existing_filepath(Archive_filepath)
    print("Building ZIP ===> ", Archive_filepath)
    patoolib.create_archive(Archive_filepath, getFilesList(_folder))
    print(f"Archived Success: {Archive_filepath}")
    return Archive_filepath

def processArchive(Archive_File):
    result_Archive = "/content/drive/MyDrive/" + os.path.split(Archive_File)[1]
    extracted_folder = extractArchive(Archive_File)
    if JUST_EXTRACT:
     return
    ffmpeg_folder = useFFmpegFolder(extracted_folder)
    createArchive(ffmpeg_folder, result_Archive)
    getSize(Archive_File)
    getSize(result_Archive)
    if Delete_Extracted_Data:
        deleteFolder(extracted_folder)
        deleteFolder(ffmpeg_folder)
    return result_Archive


# -------- Main Workflow --------

def main(filesListStr):
    drive_root = '/content/drive/MyDrive/'
    Files_List = strToList(filesListStr)

    for i, _File in enumerate(Files_List):
        if _File.startswith('http'):
            _File = Aria2Downloader(_File)
            if JUST_DOWNLOAD:
                return
        if os.path.exists(_File):
            if _File.endswith('.mkv'):
                output_file = ""
                if not _File.startswith(drive_root):
                    output_file = os.path.join(drive_root, os.path.split(_File)[1])
                useFFmpeg(_File, output_file, 1)
            elif patoolib.is_archive(_File):
                processArchive(_File)
            elif os.path.isdir(_File):
                useFFmpegFolder(_File, 1)
            else:
                print("File Exists but Unsupported File: ", _File)
        else:
            print(f"Error FILES {i+1}: {_File}")

main(FILES)

In [ ]:
#@title wget Downloader
URL = "https://mixologystreambot-6bb897237317.herokuapp.com/36695/60656c2c21894946bade447fea690315.mkv?hash=AgADlQ"  #@param {type: "string"}

!wget {URL} --no-verbose --show-progress -N

In [ ]:

#@title Save Public Google Drive Files

from google.colab import drive
from google.colab import auth
from oauth2client.client import GoogleCredentials

from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive

from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# Authenticate and initialize
auth.authenticate_user()

# Get credentials
credentials = GoogleCredentials.get_application_default()
service = build('drive', 'v3', credentials=credentials)

public_drive_urls = "https://drive.google.com/file/d/13Qdkvg8uKGuZv2_8Z1kyUzCMSSlblEEb/view?usp=drivesdk"  # @param {type: "string"}
# https://drive.google.com/file/d/13Qdkvg8uKGuZv2_8Z1kyUzCMSSlblEEb/view?usp=drivesdk
# https://drive.google.com/drive/folders/1NrBvFvL-Jro3VQBSQhHiOj4-oEEnCIUl
# https://drive.google.com/drive/folders/1glEPV-ISGauk8_HXQ6dV6AfH1HJRsVc6?usp=sharing


def strToList(listStr, separator=","):
    listStr = ' '.join(listStr.split()) # Remove extra spaces from String
    myList = [_.strip() for _ in listStr.split(separator)]
    myList = [_ for _ in myList if _ and _.strip()] # Remove empty items from list
    showAllURLs = "\n".join(f"{i+1}. {ele}" for i, ele in enumerate(myList))
    print(showAllURLs)
    return myList


def cleanURL(url):
    if not url:
        return url
    if "." in url:
        if not url.lower().startswith("http"):
            url = "http://" + url

    return url


def extractPublicDriveID(url):
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)
    try:
        # Check if the URL is a file or folder and extract the ID accordingly
        file_id = ""
        if "file/d/" in url:
            file_id = url.split('/d/')[1].split('/')[0]
        elif "folders/" in url:
            file_id = url.split('folders/')[1].split('/')[0]
        else:
            return ""  # Invalid URL format

        if "?usp=sharing" in file_id:
            file_id = file_id.replace("?usp=sharing","")

        # Create a file object and load metadata using the extracted ID
        file = drive.CreateFile({'id': file_id})
        file.FetchMetadata()  # Fetch metadata to verify ID and access permissions
        return file['id']
    except Exception as e:
        return ""

def getDriveMetadata(_id, metadata_key):
    _metadata = service.files().get(fileId=_id, fields=metadata_key).execute()
    metadata_value = _metadata.get(metadata_key)
    return metadata_value

# Function to create a folder in your Drive
def create_drive_folder(name, parent_id=None):
    folder_metadata = {
        'name': name,
        'mimeType': 'application/vnd.google-apps.folder',
        'parents': [parent_id] if parent_id else ["root"]
    }
    folder = service.files().create(body=folder_metadata, fields='id').execute()
    return folder.get('id')

# Function to copy a drive file
def transfer_drive_file(file_id, name="", parent_id=None):
    if not name:
        name = getDriveMetadata(file_id, 'name')
    copied_file_metadata = {
        'name': name,
        'parents': [parent_id] if parent_id else ["root"]
    }
    copied_file = service.files().copy(fileId=file_id, body=copied_file_metadata).execute()
    # return name, copied_file.get('id')
    return name

# Function to recursively copy a folder
def transfer_drive_folder(folder_id, parent_id=None):
    # Get the folder metadata
    folder_name = getDriveMetadata(folder_id, 'name')

    # Create the folder in your Drive
    new_folder_id = create_drive_folder(folder_name, parent_id)

    # List all items in the folder
    results = service.files().list(
        q=f"'{folder_id}' in parents",
        fields="files(id, name, mimeType)"
    ).execute()

    items = results.get('files', [])

    for item in items:
        if item['mimeType'] == 'application/vnd.google-apps.folder':
            # Recursively copy subfolder
            transfer_drive_folder(item['id'], new_folder_id)
        else:
            # Copy individual file
            transfer_drive_file(item['id'], item['name'], new_folder_id)

    print(f"Copied folder '{folder_name}' to your Drive (ID: {new_folder_id})")


def downloadFromPublicDrive(Drive_URLs):
    colab_root = "/content/"
    drive_root = "/content/drive/MyDrive/"
    file_folder_name = ""
    file_folder_path = ""

    Drive_URLs_List = strToList(Drive_URLs)
    for url in Drive_URLs_List:
        public_drive_id = extractPublicDriveID(url)
        if "file/d/" in url:
            # Start copying the folder
            try:
                file_folder_name = transfer_drive_file(public_drive_id)
                file_folder_path = drive_root + file_folder_name
            except HttpError as error:
                print(f"Drive error occurred: {error}")
        elif "folders/" in url:
            # Start copying the folder
            try:
                transfer_drive_folder(public_drive_id)
                file_folder_name = getDriveMetadata(public_drive_id, 'name')
                file_folder_path = drive_root + file_folder_name
            except HttpError as error:
                print(f"Drive error occurred: {error}")

        print(f"Drive Transfer Successfull: {file_folder_name}")


downloadFromPublicDrive(public_drive_urls)


# _. _. _.  _. _. _.  _. _. _.  _. _. _.  _. _. _.  _. _. _.

In [ ]:

#@title dex2c Encrypter

import os
from IPython.display import clear_output
root = "/content"
ndk_path = root + "/android-ndk-r27c"
dex2c_path = root + "/dex2c"
dex2c_conf_filepath = f'{dex2c_path}/dcc.cfg'
filter_filepath = f'{dex2c_path}/filter.txt'

def strToList(listStr, separator):
    listStr = ' '.join(listStr.split()) # Remove extra spaces from String
    myList = [_.strip() for _ in listStr.split(separator)]
    myList = [_ for _ in myList if _ and _.strip()] # Remove empty items from list
    return myList

def add_suffix_to_file(file_path, suffix="_Suffix"):
    parent_dir_path, file_name = os.path.split(file_path)
    name, ext = os.path.splitext(file_name)
    new_file_name = f"{name}{suffix}{ext}"
    new_file_path = os.path.join(parent_dir_path, new_file_name)
    return new_file_path

def update_file(file_path, text):
    with open(file_path, 'w') as file:
        file.write(text)

if not os.path.exists(ndk_path):
    !wget https://dl.google.com/android/repository/android-ndk-r27c-linux.zip --no-verbose --show-progress -N
    !7z x "android-ndk-r27c-linux.zip"

if not os.path.exists(dex2c_path):
    !git clone https://github.com/codehasan/dex2c && cd "{dex2c_path}" && pip install wget && pip install tqdm && wget -O tools/apktool.jar https://bitbucket.org/iBotPeaches/apktool/downloads/apktool_2.10.0.jar && pip3 install -r requirements.txt && sudo apt-get install openjdk-17-jdk zipalign

clear_output()

Input_Apk = "https://mixologystreambot-6bb897237317.herokuapp.com/36481/Via+Browser+v6.2.0+%28Mod%29.apk?hash=AgADPh"  #@param {type: "string"}
#Input_Apk = "/content/sample.apk"  #@param {type: "string"}
Ouput_Apk = ""  #@param {type: "string"}
#@markdown ***Rule#1 | Rule#2 | Rule#3 so on. ***
Filter_Rules = "mark/via/.*;.*"  #@param {type: "string"}
#Filter_Rules = ".* , com.* "  #@param {type: "string"}
Filter_Rules = '\n'.join(strToList(Filter_Rules, "|"))
# mark/via/.*;.*

dex2c_conf = f'''{{
    "apktool": "tools/apktool.jar",
    "ndk_dir": "{ndk_path}",
    "signature": {{
        "keystore_path": "keystore/debug.keystore",
        "alias": "androiddebugkey",
        "keystore_pass": "android",
        "store_pass": "android",
        "v1_enabled": true,
        "v2_enabled": true,
        "v3_enabled": true
    }}
}}'''
update_file(dex2c_conf_filepath, dex2c_conf)

filter = f'''
# Rules are made using regex patterns

# Protect all methods in all classes
# Not recommended for real world applications.
{Filter_Rules}
'''
update_file(filter_filepath, filter)

import wget
if Input_Apk.startswith(('http://', 'https://')):
    Input_Apk = wget.download(Input_Apk)
Input_Apk = os.path.abspath(Input_Apk)

if not Ouput_Apk:
    Ouput_Apk = add_suffix_to_file(Input_Apk, '_dex2c')
Ouput_Apk = os.path.abspath(Ouput_Apk)

if os.path.exists(Input_Apk):
    !cat "{dex2c_conf_filepath}" && cat "{filter_filepath}"
    !cd dex2c && python3 dcc.py -a "{Input_Apk}" -o "{Ouput_Apk}"
    print("Input")
    !du -BM "{Input_Apk}"
    print("Output")
    !du -BM "{Ouput_Apk}"
    print("\a")
else:
    print(f"Input Apk not found: {Input_Apk}")

In [ ]:

#@title APK Downloader

!pip install PyAPKDownloader

App_Package = "https://play.google.com/store/apps/details?id=mark.via.gp"  #@param {type: "string"}
App_Package = App_Package.split("id=")[1]

from PyAPKDownloader.aptoide import Aptoide
Downloader = Aptoide()

Downloader.download_by_package_name(package_name=App_Package, file_name="default", version="latest", in_background=False, limit=30)

In [ ]:
from PyAPKDownloader.apkpure import ApkPure
Downloader = ApkPure()
Downloader.download_by_package_name(package_name="com.tencent.ig", file_name="default", version="latest", app_ext="apk", in_background=False)

In [ ]:
!ls -l

In [ ]:

#@title Telegram forwarder

try:
    from pyrogram import Client, filters
except Exception as e:
    !pip install pyrogram tgCrypto

from pyrogram import Client, filters
import nest_asyncio
nest_asyncio.apply()

# Replace with your API ID, API Hash, and session name
app = Client(
    "my_account",
    api_id="22737307",  # Get from https://my.telegram.org
    api_hash="a9e79cc842e36da3e3f0b3711f8b0d5e"  # Get from https://my.telegram.org
)

# Replace with the source channel ID (from which posts will be forwarded)
SOURCE_CHANNEL_ID = "@reversalhub"  # Can be a username (str) or ID (int)

# Replace with your destination channel ID (your channel where posts will be forwarded)
DESTINATION_CHANNEL_ID = "@premiumApps000"  # Can be a username (str) or ID (int)

async def forward_all_posts():
    async with app:
        print("Fetching and forwarding posts...")

        # Fetch all messages from the source channel
        for message in app.get_chat_history(SOURCE_CHANNEL_ID):
            try:
                if message.media_group_id:
                    # Handle media groups (albums)
                    media_group = await app.get_media_group(SOURCE_CHANNEL_ID, message.id)
                    # Forward the entire media group to your channel
                    await app.forward_messages(DESTINATION_CHANNEL_ID, SOURCE_CHANNEL_ID, [m.id for m in media_group])
                    print(f"Forwarded media group: {message.id}")
                else:
                    # Handle single messages
                    await message.forward(DESTINATION_CHANNEL_ID)
                    print(f"Forwarded message: {message.id}")
            except Exception as e:
                print(f"Failed to forward message {message.id}: {e}")

# Run the function to forward all posts
await app.run(forward_all_posts())

In [ ]:
!rm -f *

rm: cannot remove 'drive': Is a directory
rm: cannot remove 'Party.Till.I.Die.S01.720p.AMZN.WEB-DL.DDP2.0.H.265-PrimeFix': Is a directory
rm: cannot remove 'sample_data': Is a directory
rm: cannot remove 'The.Roshans.S01.720p.NF.WEB-DL.AAC5.1.AV1-PrimeFix': Is a directory


In [ ]:
e
#@title MKV Merger

MKV_FILES = "https://mixologystreambot-6bb897237317.herokuapp.com/36917/%5BToonworld4all%5D+Solo+Leveling+S02E01+720p+x265+10bit+WEB-DL+.mkv?hash=AgADeB, https://mixologystreambot-6bb897237317.herokuapp.com/36923/Secret+Level+S1+2024+E9-15+Dual+Audio+Hindi+-+English+Comple.mkv?hash=AgAD4A"  # @param {type: "string"}
#MKV_FILES = "https://mixologystreambot-6bb897237317.herokuapp.com/36693/114b80ac782c4e8583c7016c9f83fe13.mkv?hash=AgADlA, https://mixologystreambot-6bb897237317.herokuapp.com/36695/60656c2c21894946bade447fea690315.mkv?hash=AgADlQ"  # @param {type: "string"}
OUTPUT_DIR = ""  #@param {type: "string"}
MAX_FileSize = None  #@param {type: "number"}
JUST_DOWNLOAD= False  #@param {type: "boolean"}
# Delete_Extracted_Data = False  #@param {type: "boolean"}
#@markdown ***Start 00:00:00 / End 00:00:10***
# trim_start = ""  #@param {type: "string"}
# trim_end = ""  #@param {type: "string"}

from IPython.display import clear_output
import subprocess
import os
import re
import requests
import patoolib
from google.colab import drive
from zipfile import ZipFile
import ffmpeg
from pymediainfo import MediaInfo
import mimetypes
# Mount Google Drive
drive.mount('/content/drive')
clear_output()


def deleteFolder(input_folder):
    !rm -rf "{input_folder}"
    print(f"Folder Deleted: {input_folder}")

def strToList(listStr, separator=","):
    listStr = ' '.join(listStr.split()) # Remove extra spaces from String
    myList = [_.strip() for _ in listStr.split(separator)]
    myList = [_ for _ in myList if _ and _.strip()] # Remove empty items from list
    showAllURLs = "\n".join(f"{i+1}. {ele}" for i, ele in enumerate(myList))
    print(showAllURLs)
    return myList

def extract_file_info(file_path):
    # parent_dir_path = os.path.dirname(file_path)
    # file_name = os.path.basename(file_path)
    parent_dir_path, file_name = os.path.split(file_path)
    _name, _ext = os.path.splitext(file_name)
    return parent_dir_path,  file_name, _name, _ext[1:]

def getMKVFilesList(folder_path, relative=0):
    all_files = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.mkv'):
                full_path = os.path.join(root, file)
                if relative == 1:
                    full_path = os.path.relpath(full_path, folder_path)
                all_files.append(full_path)

    all_files.sort()
    return all_files


def extractArchive(Archive_File, extracted_folder=''):
    if not extracted_folder:
        extracted_folder = os.path.splitext(Archive_File)[0]
    if os.path.exists(extracted_folder):
        deleteFolder(extracted_folder)
    os.makedirs(extracted_folder, exist_ok=True)
    print(f"\nExtracting archive to: {extracted_folder}")
    patoolib.extract_archive(Archive_File, outdir=extracted_folder)
    print(f"Extraction complete: {extracted_folder}\n")
    return extracted_folder


def runCommand(cmd, showLog=0):
    if showLog == 1:
        # Print logs in real-time, if 2nd param is 1
        print(f"Running command: {cmd}")
        process = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
        for line in iter(process.stdout.readline, b''):
            print(line.decode("utf-8"), end="")

        process.stdout.close()
        process.wait()
    else:
        process = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    return process.returncode


def getResponseFileName(url):
    # Define the maximum filename length (commonly 255 characters)
    MAX_FILENAME_LENGTH = 255

    # Get the response from the URL
    response = requests.get(url, stream=True)

    # Extract the filename from the Content-Disposition header
    filename = response.headers.get('content-disposition', '')
    if 'filename=' in filename:
        filename = filename.split("filename=")[-1].strip('"')
    else:
        return ""

    # Define a regex pattern to match unsuitable characters for filenames
    invalid_chars = r'[<>:"/\\|?*\']'

    # Replace unsuitable characters with a hyphen
    valid_filename = re.sub(invalid_chars, '-', filename)

    # Truncate the filename if it exceeds the maximum length
    if len(valid_filename) > MAX_FILENAME_LENGTH:
        # Preserve the file extension if present
        if '.' in valid_filename:
            name, extension = valid_filename.rsplit('.', 1)
            truncated_name = name[:MAX_FILENAME_LENGTH - len(extension) - 1]  # Reserve space for the dot
            valid_filename = f"{truncated_name}.{extension}"
        else:
            valid_filename = valid_filename[:MAX_FILENAME_LENGTH]

    return valid_filename


def Aria2Downloader(url, output_file=''):
    if not url.startswith("http"):
        print(f"Invalid URL: {url}")
    _responseFileName = getResponseFileName(url)
    if _responseFileName:
        if not output_file:
            output_file = _responseFileName
        if os.path.exists(output_file):
            print(f"Download File Already Exists: {output_file}")
            return output_file
        else:
            """Download a file using Aria2."""
            print(f"Downloading file: {url}")
            command = f"aria2c -x 16 -s 16 -o \"{output_file}\" {url}"
            status_code = runCommand(command, 1)
            if status_code == 0:
                print(f"\nDownload Successfull: {output_file}\n")
                return output_file
            else:
                print(f"Download Error Status code({status_code}): {url}")
    else:
        print(f"Download Error Response FileName: {url}")
    return url


def mergeMKVs(video_paths, output_dir, max_size_gb):
    if len(video_paths) < 2:
        print(video_paths, f" ==> Can't Merge {len(video_paths)} video")
        return video_paths

    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Function to get track IDs using ffmpeg probe
    def get_track_ids(file):
        probe = ffmpeg.probe(file)
        return {stream['index'] for stream in probe['streams']}

    # Get all track IDs across files
    all_track_ids = set.union(*[get_track_ids(file) for file in video_paths])

    # Find files with missing track IDs
    files_to_exclude = []
    valid_files = []

    for file in video_paths:
        file_track_ids = get_track_ids(file)
        if all_track_ids.issubset(file_track_ids):
            valid_files.append(file)
        else:
            files_to_exclude.append(file)

    # If no valid files are left, exit
    if not valid_files:
        print("No valid files to merge. All files have missing track IDs.")
        return files_to_exclude

    # Base output file path
    base_output_path = os.path.join(output_dir, extract_file_info(valid_files[0])[2] + "_merged")

    # Convert max_size_gb to bytes (mkvmerge uses bytes for splitting)
    max_size_bytes = max_size_gb * 1024 * 1024 * 1024

    command = [
        "mkvmerge",
        "-o", f"{base_output_path}.mkv",
        "--split", f"{max_size_bytes}",
    ]

    # Append valid videos sequentially
    for i, video_path in enumerate(valid_files):
        if i == 0:
            command.append(video_path)  # First video
        else:
            command.extend(["+", video_path])  # Append subsequent videos

    # Run the mkvmerge command
    command_str = ' '.join(command)
    runCommand(command_str, 1)

    # Display files that were not merged
    if files_to_exclude:
        print("The following files were not merged due to missing track IDs:")
        for file in files_to_exclude:
            print(f"- {file}")
    else:
        print("All files were merged successfully.")

    return files_to_exclude


def main(filesListStr, output_dir=''):
    Files_List = strToList(filesListStr, ",")
    _File = Files_List[0]

    if len(Files_List) == 0 or (len(Files_List) == 1 and os.path.exists(_File) and _File.endswith('.mkv')):
        print(Files_List, " ==> Give At Least 2 files for Merging")
        exit()

    if len(Files_List) == 1:
        _File = Files_List[0]
        if _File.startswith('http'):
            _File = Aria2Downloader(_File)
        if os.path.exists(_File):
            if patoolib.is_archive(_File):
                _Folder = extractArchive(_File)
                Files_List = getMKVFilesList(_Folder)
            elif os.path.isdir(_File):
                Files_List = getMKVFilesList(_File)

    valid_Files_List = []
    if len(Files_List) > 1:
        for i, valid_File in enumerate(Files_List):
            if valid_File.startswith('http'):
                valid_File = Aria2Downloader(valid_File)
            if os.path.exists(valid_File):
                if valid_File.endswith('.mkv'):
                    valid_File = os.path.abspath(valid_File)
                    valid_Files_List.append(valid_File)
                else:
                    print("File Exists but Unsupported File Format: ", valid_File)
            else:
                print(f"Error FILES {i+1}: {valid_File}")
                exit()
        print("Valid Files are:\n", "\n".join(valid_Files_List))

    if len(valid_Files_List) > 1:
        if JUST_DOWNLOAD:
            exit()
        if not output_dir:
            output_dir = '/content/drive/MyDrive'
        mergeMKVs(valid_Files_List, output_dir)

main(MKV_FILES, OUTPUT_DIR, MAX_FileSize)

In [10]:
from pymediainfo import MediaInfo

def get_track_info(file_path):
    """
    Fetches and prints track details (ID, Type, Language, Title) using pymediainfo.

    Args:
        file_path (str): Path to the media file.
    """
    try:
        media_info = MediaInfo.parse(file_path)
        for track in media_info.tracks:
            if track.track_type in ["Video", "Audio", "Text"]:  # Only show relevant tracks
                track_id = track.track_id or "Unknown"
                track_type = track.track_type or "Unknown"
                language = track.language or "Unknown"
                title = track.title or "Untitled"

                print(f"ID: {track_id} | Type: {track_type} | Language: {language} | Title: {title}")
    except Exception as e:
        print(f"Error occurred: {e}")

#
# Example usage
Video = "/content/The.Roshans.S01.720p.NF.WEB-DL.AAC5.1.AV1-PrimeFix/The.Roshans.S01E02.720p.NF.WEB-DL.AAC5.1.AV1-PrimeFix.mkv"  #@param {type: "string"}
get_track_info(Video)

ID: 1 | Type: Video | Language: Unknown | Title: Untitled
ID: 2 | Type: Audio | Language: hi | Title: Untitled
ID: 3 | Type: Text | Language: en | Title: Untitled
ID: 4 | Type: Text | Language: en | Title: Untitled
ID: 5 | Type: Text | Language: en | Title: SDH
